In [33]:
import re
import ast
import zipfile
import openpyxl
import pandas as pd
import xml.etree.ElementTree as ET
from collections import defaultdict


In [1]:
###############################################################################################################################################################################################################################################################################################
##################################################################################################################################################### PHASE 1 #################################################################################################################################
###############################################################################################################################################################################################################################################################################################






class EXCEL_XML_EXTRACTOR:
    def __init__(self, FILE_PATH):
        self.FILE_PATH = FILE_PATH
        self.XML_FILES = self.EXTRACT_XML_FILES()

    def EXTRACT_XML_FILES(self):
        XML_FILES = {}
        with zipfile.ZipFile(self.FILE_PATH, 'r') as ZIP_REF:
            for FILE in ZIP_REF.namelist():
                if FILE.endswith('.xml'):
                    with ZIP_REF.open(FILE) as f:
                        XML_FILES[FILE] = f.read()
        return XML_FILES







class WORKBOOK_PARSING:
    def __init__(self, XML_FILES):
        self.XML_FILES = XML_FILES
        self.DICT = self.PARSE_WORKBOOK()

    def PARSE_WORKBOOK(self):
        MAP = {}
        if 'xl/workbook.xml' in self.XML_FILES:
            ROOT = ET.fromstring(self.XML_FILES['xl/workbook.xml'])
            SHEETS = ROOT.findall('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}sheet')
            for i, SHEET in enumerate(SHEETS):
                NAME = SHEET.attrib.get('name')
                STATE = SHEET.attrib.get('state', 'visible')
                MAP[f'sheet{i + 1}'] = {'NAME': NAME, 'STATE': STATE}
        return MAP









class WORKSHEET_PARSING:
    def __init__(self, XML_FILES, SHEET_NAME_MAP):
        self.XML_FILES = XML_FILES
        self.SHEET_NAME_MAP = SHEET_NAME_MAP
        self.DICT = self.PARSE_WORKSHEETS()

    def PARSE_WORKSHEETS(self):
        WORKSHEETS = {}
        for FILE_NAME, XML_CONTENT in self.XML_FILES.items():
            if 'xl/worksheets/sheet' in FILE_NAME:
                ROOT = ET.fromstring(XML_CONTENT)
                SHEET_DATA = {}

                for CELL in ROOT.findall('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}c'):
                    CELL_REF = CELL.attrib.get('r')
                    CELL_TYPE = CELL.attrib.get('t')
                    STYLE_INDEX = CELL.attrib.get('s')
                    FORMULA = CELL.find('{http://schemas.openxmlformats.org/spreadsheetml/2006/main}f')
                    VALUE = CELL.find('{http://schemas.openxmlformats.org/spreadsheetml/2006/main}v')

                    VALUE_TEXT = VALUE.text if VALUE is not None else None

                    CELL_DATA = {
                        'TYPE': CELL_TYPE,
                        'STYLE_INDEX': STYLE_INDEX,
                        'FORMULA': FORMULA.text if FORMULA is not None else None,
                        'VALUE': VALUE_TEXT
                    }

                    SHEET_DATA[CELL_REF] = CELL_DATA

                # Extract the sheet internal reference (e.g., 'sheet1')
                SHEET_NAME_KEY = FILE_NAME.split('/')[-1].replace('.xml', '')
                # Map the internal sheet name to the user-defined sheet name
                USER_DEFINED_NAME = self.SHEET_NAME_MAP.get(SHEET_NAME_KEY, {}).get('NAME', SHEET_NAME_KEY)
                WORKSHEETS[USER_DEFINED_NAME] = SHEET_DATA

        return WORKSHEETS





class CALC_CHAIN_PARSING:
    def __init__(self, XML_FILES, SHEET_NAME_MAP):
        self.XML_FILES = XML_FILES
        self.SHEET_NAME_MAP = SHEET_NAME_MAP
        self.DICT = self.CALC_CHAIN_PARSE()

    def CALC_CHAIN_PARSE(self):
        CALC_CHAIN = []
        if 'xl/calcChain.xml' in self.XML_FILES:
            ROOT = ET.fromstring(self.XML_FILES['xl/calcChain.xml'])
            for CELL in ROOT.findall('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}c'):
                CELL_REF = CELL.attrib.get('r')
                SHEET_ID = CELL.attrib.get('i')
                # Map SHEET_ID to the user-defined sheet name using SHEET_NAME_MAP
                SHEET_NAME = self.SHEET_NAME_MAP.get(f'sheet{SHEET_ID}', {}).get('NAME', f'sheet{SHEET_ID}')
                CALC_CHAIN.append({'CELL_REF': CELL_REF, 'SHEET_NAME': SHEET_NAME})
        return CALC_CHAIN





class SHARED_STRING_PARSING:

    def __init__(self, XML_FILES):
        self.XML_FILES = XML_FILES
        self.DICT = self.SHARED_STRINGS_PARSE()

    def SHARED_STRINGS_PARSE(self):
        SHARED_STRINGS = {}
        if 'xl/sharedStrings.xml' in self.XML_FILES:
            ROOT = ET.fromstring(self.XML_FILES['xl/sharedStrings.xml'])
            for i, SI in enumerate(ROOT.findall('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}si')):
                TEXT_NODE = SI.find('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}t')
                if TEXT_NODE is not None:
                    SHARED_STRINGS[i] = TEXT_NODE.text
        return SHARED_STRINGS








class STYLES_PARSING:
    
    def __init__(self, XML_FILES):
        self.XML_FILES = XML_FILES
        self.DICT = self.STYLE_PARSE()

    def STYLE_PARSE(self):
        STYLES = {  
            'NUMBER_FORMATS': {},
            'FONTS': [],
            'FILLS': [],
            'BORDERS': [],
            'CELLXFS': []
        }
        
        if 'xl/styles.xml' in self.XML_FILES:
            ROOT = ET.fromstring(self.XML_FILES['xl/styles.xml'])

            # 1. Extract number formats
            NUM_FMTS = ROOT.find('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}numFmts')
            if NUM_FMTS is not None:
                for NUM_FMT in NUM_FMTS.findall('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}numFmt'):
                    FMT_ID = NUM_FMT.attrib.get('numFmtId')
                    FORMAT_CODE = NUM_FMT.attrib.get('formatCode')
                    STYLES['NUMBER_FORMATS'][FMT_ID] = FORMAT_CODE

            # 2. Extract fonts
            FONTS = ROOT.find('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}fonts')
            if FONTS is not None:
                for FONT in FONTS.findall('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}font'):
                    FONT_DATA = {
                        'NAME': FONT.find('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}name').attrib.get('val'),
                        'SIZE': FONT.find('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}sz').attrib.get('val'),
                        'BOLD': FONT.find('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}b') is not None,
                        'ITALIC': FONT.find('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}i') is not None,
                        'COLOR': FONT.find('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}color').attrib.get('rgb') if FONT.find('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}color') is not None else None
                    }
                    STYLES['FONTS'].append(FONT_DATA)

            # 3. Extract fills
            FILLS = ROOT.find('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}fills')
            if FILLS is not None:
                for FILL in FILLS.findall('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}fill'):
                    PATTERN_FILL = FILL.find('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}patternFill')
                    FILL_DATA = {
                        'PATTERN_TYPE': PATTERN_FILL.attrib.get('patternType') if PATTERN_FILL is not None else None,
                        'FG_COLOR': PATTERN_FILL.find('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}fgColor').attrib.get('rgb') if PATTERN_FILL is not None and PATTERN_FILL.find('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}fgColor') is not None else None,
                        'BG_COLOR': PATTERN_FILL.find('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}bgColor').attrib.get('rgb') if PATTERN_FILL is not None and PATTERN_FILL.find('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}bgColor') is not None else None
                    }
                    STYLES['FILLS'].append(FILL_DATA)

            # 4. Extract borders
            BORDERS = ROOT.find('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}borders')
            if BORDERS is not None:
                for BORDER in BORDERS.findall('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}border'):
                    BORDER_DATA = {
                        'LEFT': BORDER.find('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}left') is not None,
                        'RIGHT': BORDER.find('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}right') is not None,
                        'TOP': BORDER.find('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}top') is not None,
                        'BOTTOM': BORDER.find('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}bottom') is not None
                    }
                    STYLES['BORDERS'].append(BORDER_DATA)

            # 5. Extract cellXfs (Cell formats)
            CELLXFS = ROOT.find('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}cellXfs')
            if CELLXFS is not None:
                for XF in CELLXFS.findall('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}xf'):
                    XF_DATA = {
                        'NUM_FMT_ID': XF.attrib.get('numFmtId'),
                        'FONT_ID': XF.attrib.get('fontId'),
                        'FILL_ID': XF.attrib.get('fillId'),
                        'BORDER_ID': XF.attrib.get('borderId')
                    }
                    STYLES['CELLXFS'].append(XF_DATA)

        return STYLES









class DRAWING_PARSING:

    def __init__(self, XML_FILES):
        self.XML_FILES = XML_FILES
        self.DICT = self.DRAWING_PARSE()

    def DRAWING_PARSE(self):
        DRAWINGS = defaultdict(list)
        for FILE_NAME, XML_CONTENT in self.XML_FILES.items():
            if 'xl/drawings/drawing' in FILE_NAME:
                ROOT = ET.fromstring(XML_CONTENT)
                DRAWING_INFO = []

                # Look for twoCellAnchor elements
                for ANCHOR in ROOT.findall('.//{http://schemas.openxmlformats.org/drawingml/2006/spreadsheetDrawing}twoCellAnchor'):
                    
                    # Identify images
                    BLIP = ANCHOR.find('.//{http://schemas.openxmlformats.org/drawingml/2006/main}blip')
                    if BLIP is not None:
                        EMBED = BLIP.attrib.get('{http://schemas.openxmlformats.org/officeDocument/2006/relationships}embed')
                        DRAWING_INFO.append({'TYPE': 'IMAGE', 'EMBED_ID': EMBED})

                    # Identify shapes (e.g., rectangles, circles)
                    SP = ANCHOR.find('.//{http://schemas.openxmlformats.org/drawingml/2006/main}sp')
                    if SP is not None:
                        SP_NAME = SP.find('.//{http://schemas.openxmlformats.org/drawingml/2006/main}nvSpPr/{http://schemas.openxmlformats.org/drawingml/2006/main}cNvPr').attrib.get('name', 'Shape')
                        DRAWING_INFO.append({'TYPE': 'SHAPE', 'NAME': SP_NAME})

                    # Identify charts
                    GRAPHIC_FRAME = ANCHOR.find('.//{http://schemas.openxmlformats.org/drawingml/2006/main}graphicFrame')
                    if GRAPHIC_FRAME is not None:
                        CHART = GRAPHIC_FRAME.find('.//{http://schemas.openxmlformats.org/drawingml/2006/chart}chart')
                        if CHART is not None:
                            CHART_ID = CHART.attrib.get('{http://schemas.openxmlformats.org/officeDocument/2006/relationships}id')
                            DRAWING_INFO.append({'TYPE': 'CHART', 'CHART_ID': CHART_ID})

                    # Identify connectors (e.g., lines)
                    CNX_SP = ANCHOR.find('.//{http://schemas.openxmlformats.org/drawingml/2006/main}cxnSp')
                    if CNX_SP is not None:
                        CNX_NAME = CNX_SP.find('.//{http://schemas.openxmlformats.org/drawingml/2006/main}nvCxnSpPr/{http://schemas.openxmlformats.org/drawingml/2006/main}cNvPr').attrib.get('name', 'Connector')
                        DRAWING_INFO.append({'TYPE': 'CONNECTOR', 'NAME': CNX_NAME})

                DRAWINGS[FILE_NAME] = DRAWING_INFO
        return DRAWINGS



class TABLES_PARSING:

    def __init__(self, XML_FILES):
        self.XML_FILES = XML_FILES
        self.DICT = self.TABLES_PARSE()

    def TABLES_PARSE(self):

        TABLES = {}
        for FILE_NAME, XML_CONTENT in self.XML_FILES.items():
            if 'xl/tables/table' in FILE_NAME:
                ROOT = ET.fromstring(XML_CONTENT)
                TABLE_INFO = {}
                TABLE_INFO['NAME']      = ROOT.attrib.get('name')
                TABLE_INFO['REF']       = ROOT.attrib.get('ref')  # Table range (e.g., A1:B10)
                TABLE_INFO['COLUMNS']   = [col.attrib.get('name') for col in ROOT.findall('.//{http://schemas.openxmlformats.org/spreadsheetml/2006/main}tableColumn')]
                TABLES[FILE_NAME]       = TABLE_INFO
        return TABLES
    


class THEME_PARSING:

    def __init__(self, XML_FILES):
        self.XML_FILES = XML_FILES
        self.DICT = self.THEME_PARSE()

    def THEME_PARSE(self):

        THEME = {}
        if 'xl/theme/theme1.xml' in self.XML_FILES:  
            ROOT = ET.fromstring(self.XML_FILES['xl/theme/theme1.xml'])
            COLOUR_SCHEME = []
            for COLOUR in ROOT.findall('.//{http://schemas.openxmlformats.org/drawingml/2006/main}clrScheme//{http://schemas.openxmlformats.org/drawingml/2006/main}srgbClr'):
                COLOUR_SCHEME.append(COLOUR.attrib.get('val'))
            THEME['color_scheme'] = COLOUR_SCHEME
        return THEME




class RELATIVE_SHEETS_PARSING:
    
    def __init__(self, XML_FILES, SHEET_NAME_MAP):
        self.XML_FILES = XML_FILES
        self.SHEET_NAME_MAP = SHEET_NAME_MAP
        self.DICT = self.parse_sheet_rels()

    def parse_sheet_rels(self):
        SHEET_RELS = defaultdict(list)
        
        for FILE_NAME, XML_CONTENT in self.XML_FILES.items():
            if 'xl/worksheets/_rels/sheet' in FILE_NAME and '.rels' in FILE_NAME:
                ROOT = ET.fromstring(XML_CONTENT)

                SHEET_NUM_KEY = FILE_NAME.split('/')[-1].replace('.xml.rels', '')
                USER_DEFINED_NAME = self.SHEET_NAME_MAP.get(SHEET_NUM_KEY, {}).get('NAME', SHEET_NUM_KEY)

                for REL in ROOT.findall('.//{http://schemas.openxmlformats.org/package/2006/relationships}Relationship'):
                    REL_ID = REL.attrib.get('Id')
                    TARGET = REL.attrib.get('Target')
                    REL_TYPE = REL.attrib.get('Type').split('/')[-1].upper()  # Capitalize the TYPE
                    SHEET_RELS[USER_DEFINED_NAME].append({'ID': REL_ID, 'TARGET': TARGET, 'TYPE': REL_TYPE})

        return SHEET_RELS
    



class CONTENT_TYPE_PARSING:

    def __init__(self, XML_FILES):
        self.XML_FILES = XML_FILES
        self.DICT = self.CONTENT_TYPE_PARSE()

    def CONTENT_TYPE_PARSE(self):

        CONTENT_TYPES = []
        if '[Content_Types].xml' in self.XML_FILES:
            ROOT = ET.fromstring(self.XML_FILES['[Content_Types].xml'])
            for OVERRIDE in ROOT.findall('.//{http://schemas.openxmlformats.org/package/2006/content-types}Override'):
                PART_NAME       = OVERRIDE.attrib.get('PartName')
                CONTENT_TYPE    = OVERRIDE.attrib.get('ContentType')
                CONTENT_TYPES.append({'PART_NAME': PART_NAME, 'CONTENT_TYPE': CONTENT_TYPE})
        return CONTENT_TYPES





class RELATIVE_WORKBOOK_PARSING:
    
    def __init__(self, XML_FILES):
        self.XML_FILES = XML_FILES
        self.DICT = self.WORKBOOK_REL_PARSE()

    def WORKBOOK_REL_PARSE(self):

        WORKBOOK_RELS = []
        if 'xl/_rels/workbook.xml.rels' in self.XML_FILES:
            ROOT = ET.fromstring(self.XML_FILES['xl/_rels/workbook.xml.rels'])
            for REL in ROOT.findall('.//{http://schemas.openxmlformats.org/package/2006/relationships}Relationship'):
                REL_ID      = REL.attrib.get('Id')
                TARGET      = REL.attrib.get('Target')
                REL_TYPE    = REL.attrib.get('Type').split('/')[-1]
                WORKBOOK_RELS.append({'ID': REL_ID, 'TARGET': TARGET, 'TYPE': REL_TYPE})
        return WORKBOOK_RELS





class EXCEL_DATA_PARSER:


    def __init__(self, FILE_PATH):
        self.XML_EXTRACTOR      = EXCEL_XML_EXTRACTOR(FILE_PATH)
        self.SHEET_NAMES        = WORKBOOK_PARSING(self.XML_EXTRACTOR.XML_FILES)
        self.WORKSHEET_PARSER   = WORKSHEET_PARSING(self.XML_EXTRACTOR.XML_FILES, self.SHEET_NAMES.DICT)
        self.CALC_CHAIN         = CALC_CHAIN_PARSING(self.XML_EXTRACTOR.XML_FILES, self.SHEET_NAMES.DICT)
        self.SHEET_RELS         = RELATIVE_SHEETS_PARSING(self.XML_EXTRACTOR.XML_FILES, self.SHEET_NAMES.DICT)
        self.SHARED_STRINGS     = SHARED_STRING_PARSING(self.XML_EXTRACTOR.XML_FILES)
        self.STYLES             = STYLES_PARSING(self.XML_EXTRACTOR.XML_FILES)
        self.DRAWINGS           = DRAWING_PARSING(self.XML_EXTRACTOR.XML_FILES)
        self.TABLES             = TABLES_PARSING(self.XML_EXTRACTOR.XML_FILES)
        self.THEME              = THEME_PARSING(self.XML_EXTRACTOR.XML_FILES)
        self.CONTENT_TYPE       = CONTENT_TYPE_PARSING(self.XML_EXTRACTOR.XML_FILES)
        self.WORKBOOK_RELS      = RELATIVE_WORKBOOK_PARSING(self.XML_EXTRACTOR.XML_FILES)


    def GET_DATA(self):

        return {'WORKSHEETS'        : self.WORKSHEET_PARSER.DICT,
                'SHEET_NAME_MAP'    : self.SHEET_NAMES.DICT,
                'SHARED_STRINGS'    : self.SHARED_STRINGS.DICT,
                'STYLES'            : self.STYLES.DICT,
                'CALC_CHAIN'        : self.CALC_CHAIN.DICT,
                'DRAWINGS'          : self.DRAWINGS.DICT,
                'TABLES'            : self.TABLES.DICT,
                'SHEET_RELS'        : self.SHEET_RELS.DICT,
                'THEME'             : self.THEME.DICT,
                'CONTENT_TYPES'     : self.CONTENT_TYPE.DICT,
                'WORKBOOK_RELS'     : self.WORKBOOK_RELS.DICT}




In [2]:



FILE_CHOICE             = 0
FILE_NAME               = ['EXCL_TEST.xlsx', 'AST_Final__2024_07_09.xlsm', 'ESG EET  I  3SC Pride Fund  I  2023 12.xlsm']
ROOT                    = r'/Users/westhomas/Desktop/ALFRED/1__REPORT_AUTOMATION/REFERENCE_FILES/'
FILE_PATH               = ROOT + '/' + FILE_NAME[FILE_CHOICE]


EXCEL_PARSED            = EXCEL_DATA_PARSER(FILE_PATH)
PARSED_DATA             = EXCEL_PARSED.GET_DATA()
WORKSHEETS              = PARSED_DATA['WORKSHEETS']



In [69]:
class EXTRACT_FORMULAS:
    def __init__(self, WORKSHEETS):
        self.WORKSHEETS = WORKSHEETS
        self.DICT       = self.EXTRACT()

    def EXTRACT(self):

        FORMULAS = {}
        for SHEET_NAME, CELLS in self.WORKSHEETS.items():
            SHEET_FORMULAS = {}
            for CELL_REF, CELL_DATA in CELLS.items():
                if 'FORMULA' in CELL_DATA and CELL_DATA['FORMULA'] is not None:
                    FORMULA = CELL_DATA['FORMULA']
                    if any(err in FORMULA for err in ['#DIV/0!', '#VALUE!', '#REF!', '#NAME?', '#N/A']):    SHEET_FORMULAS[CELL_REF] = {'FORMULA': FORMULA, 'ERROR': True}
                    else:                                                                                   SHEET_FORMULAS[CELL_REF] = {'FORMULA': FORMULA, 'ERROR': False}

            FORMULAS[SHEET_NAME] = SHEET_FORMULAS

        return FORMULAS
    

class CROSS_SHEET_REFERENCE_IDENTIFIER:
    def __init__(self, FORMULAS):
        self.FORMULAS = FORMULAS
        self.DICT = self.IDENTIFY()

    def IDENTIFY(self):
        CROSS_REFERENCES = {}

        # Regex pattern to identify sheet and cell references
        PATTERN = r'(\w+!)?([A-Z]+[0-9]+|[A-Z]+:[A-Z]+)'

        for SHEET_NAME, SHEET_FORMULAS in self.FORMULAS.items():
            # Initialize the sheet's reference list in CROSS_REFERENCES
            CROSS_REFERENCES[SHEET_NAME] = {}

            for CELL_REF, FORMULA_INFO in SHEET_FORMULAS.items():
                FORMULA = FORMULA_INFO['FORMULA']
                MATCHES = re.findall(PATTERN, FORMULA)
                
                if MATCHES:
                    REFERENCES = []
                    for match in MATCHES:
                        if match[0]:  # If the match contains a sheet name
                            REFERENCES.append({'SHEET': match[0].rstrip('!'), 'CELL': match[1]})
                        else:  # Intra-sheet reference; use the current SHEET_NAME
                            REFERENCES.append({'SHEET': SHEET_NAME, 'CELL': match[1]})
                    
                    # Add the references to the specific cell in CROSS_REFERENCES
                    CROSS_REFERENCES[SHEET_NAME][CELL_REF] = REFERENCES

        return CROSS_REFERENCES
    




In [74]:



class FORMULA_ANALYZER:
    def __init__(self, WORKSHEETS, FUNC_PATH):
        self.WORKSHEETS = WORKSHEETS
        self.FUNC_PATH  = FUNC_PATH


    def ANALYSE_FORMULAS(self):
        """
        Loop through each sheet and cell to find and parse formulas.
        Adds a breakdown of each formula to the cell data.
        """

        FUNCTIONS_DF, OPERATORS_DF = self.SOURCE_REFERENCE_INFORMATION(self.FUNC_PATH)

        for SHEET_NAME, CELLS in self.WORKSHEETS.items():
            for CELL_REFS, CELL_DATA in CELLS.items():
                FORMULA = CELL_DATA.get('FORMULA')
                if FORMULA:  
                    FORMULA_BREAKDOWN = self.FORMULA_CONVERSION(('='+FORMULA), OPERATORS_DF, FUNCTIONS_DF)
                    CELL_DATA['FORMULA_BREAKDOWN'] = FORMULA_BREAKDOWN



    def LOAD_TEST_DATA(self, EXCEL_FUNCS_PATH):
        TEST_WB                             = openpyxl.load_workbook(EXCEL_FUNCS_PATH)
        TEST_DICT_WB                        = {'SHEETS': {}, 'DATAFRAME': {}, 'COLUMN_MAP': {}}

        for idx, TEST_SHEET_NAME in enumerate(TEST_WB.sheetnames):
            TEST_WS                         = TEST_WB[TEST_SHEET_NAME]
            TEST_DF                         = pd.DataFrame(list(TEST_WS.values)[1:], columns=list(TEST_WS.values)[0])
            COLUMN_MAPPING_DF               = pd.DataFrame({'LETTER' : [openpyxl.utils.get_column_letter(i + 1) for i in range(TEST_WS.max_column)],
                                                            'HEADER' : list(TEST_WS.values)[0]})

            TEST_DICT_WB['SHEETS'][idx]     = TEST_SHEET_NAME
            TEST_DICT_WB['DATAFRAME'][idx]  = TEST_DF
            TEST_DICT_WB['COLUMN_MAP'][idx] = COLUMN_MAPPING_DF

        return TEST_DICT_WB



    def SOURCE_REFERENCE_INFORMATION(self, EXCEL_FUNCS_PATH):

        FUNC_DATA                       = self.LOAD_TEST_DATA(EXCEL_FUNCS_PATH)
        FUNCTIONS_DF                    = FUNC_DATA['DATAFRAME'][0]
        OPERATORS_DF                    = FUNC_DATA['DATAFRAME'][1]
        FUNCTIONS_DF['JSON_FIELDS']     = FUNCTIONS_DF['JSON_FIELDS'].apply(ast.literal_eval)

        return FUNCTIONS_DF, OPERATORS_DF



    def FORMULA_CONVERSION(self, FORMULA, OPERATORS_DF, FUNCTIONS_DF):

        FORMULA_LIST        = self.EXTRACTION(FORMULA)
        FORMULA_CLEAN       = self.APPLY_OPERATORS(FORMULA, OPERATORS_DF)        
        RESULT              = self.PROCESS_FORMULA(FORMULA_CLEAN[1:], FUNCTIONS_DF, FORMULA_LIST)

        return RESULT



    def EXTRACTION(self, FORMULA):
        FORMULA = FORMULA.strip()

        PATTERN                 = r"([A-Z]+)\("
        FUNCTION_LIST           = re.findall(PATTERN, FORMULA)

        if not FUNCTION_LIST:   return ['HARDCODED']
        return FUNCTION_LIST




    def APPLY_OPERATORS(self, REFERENCE, OPERATORS_DF):

        INITIAL_EQUALS                  = REFERENCE.startswith('=')
        if INITIAL_EQUALS: REFERENCE    = REFERENCE[1:] 


        for idx, ROW in OPERATORS_DF.iterrows():
            EXCEL_OP                    = re.escape(ROW['EXCEL_OPERATOR'])
            PLACEHOLDER                 = ROW['PLACEHOLDER']
            REFERENCE                   = REFERENCE.replace(EXCEL_OP, PLACEHOLDER)
        

        for idx, ROW in OPERATORS_DF.iterrows():
            PLACEHOLDER                 = ROW['PLACEHOLDER']
            PYTHON_OP                   = ROW['PYTHON_OPERATOR']
            REFERENCE                   = REFERENCE.replace(PLACEHOLDER, PYTHON_OP)
        
        if INITIAL_EQUALS: REFERENCE    = '=' + REFERENCE

        return REFERENCE




    def PROCESS_FORMULA(self, FORMULA, FUNCTIONS_DF, FORMULA_LIST):

        TOP_FUNC_NAME, TOP_FUNC_ARGS    = self.PARSE_FUNCTIONS(FORMULA)
        if 'HARDCODED' in FORMULA_LIST: return self.JSON_OUTPUT_HARDCODED(FORMULA.strip('=').strip())
        if TOP_FUNC_NAME not in FUNCTIONS_DF['EXCEL_FUNCTION'].values:
            return {"FUNCTION"      : "UNKNOWN",
                    "COMPONENTS"    : { "FUNCTION_NAME" : TOP_FUNC_NAME,
                                        "ARGUMENTS"     : TOP_FUNC_ARGS}}
        
        if not TOP_FUNC_NAME:           return {}

        return self.JSON_OUTPUT(TOP_FUNC_NAME, TOP_FUNC_ARGS, FUNCTIONS_DF)


    def JSON_OUTPUT_HARDCODED(self, value):
        return {"FUNCTION"      : "HARDCODED",
                "COMPONENTS"    : {"STRING" : value}
                }



    def PARSE_FUNCTIONS(self, FORMULA):
        PATTERN             = r"([A-Z]+)\((.*)\)"
        MATCH               = re.match(PATTERN, FORMULA.strip())
        
        if MATCH:
            FUNCTION_NAME   = MATCH.group(1)
            ARGUMENT_STR    = MATCH.group(2).strip()
            
            ARGUMENTS       = self.SPLIT_ARGUMENTS(ARGUMENT_STR)
            
            return FUNCTION_NAME, ARGUMENTS
        
        return None, []




    def JSON_OUTPUT(self, FUNCTION_NAME, ARGUMENTS, FUNCTIONS_DF):
        JSON_FIELDS                                             = (FUNCTIONS_DF[FUNCTIONS_DF['EXCEL_FUNCTION']==FUNCTION_NAME].reset_index().iloc[:,1:]).at[0, 'JSON_FIELDS']

        if len(ARGUMENTS) == len(JSON_FIELDS):
            COMPONENTS = {}
            
            for i in range(len(ARGUMENTS)):
                NESTED_FUNCS_MATCH                              = re.match(r"([A-Z]+)\((.*)\)", ARGUMENTS[i].strip())
                
                if NESTED_FUNCS_MATCH:
                    NESTED_FUNCS_NAME                           = NESTED_FUNCS_MATCH.group(1)
                    NESTED_FUNCS_ARGS                           = self.SPLIT_ARGUMENTS(NESTED_FUNCS_MATCH.group(2))
                    COMPONENTS[JSON_FIELDS[i]]                  = self.JSON_OUTPUT(NESTED_FUNCS_NAME, NESTED_FUNCS_ARGS, FUNCTIONS_DF)

                else:
                    if JSON_FIELDS[i] == "CONDITION":           COMPONENTS[JSON_FIELDS[i]] = self.PROCESS_CONDITIONS(ARGUMENTS[i].strip())
                    else:                                       COMPONENTS[JSON_FIELDS[i]] = self.NONE_PREFIX(ARGUMENTS[i].strip())
            
            return {"FUNCTION": FUNCTION_NAME, "COMPONENTS": COMPONENTS}
        
        else:    
            return {"FUNCTION": FUNCTION_NAME, "COMPONENTS": ARGUMENTS}



    def SPLIT_ARGUMENTS(self, ARGUMENT_STR):
        ARGS, CURRENT_ARG, PARENTHESIS_COUNT    = [], "", 0

        for CHAR in ARGUMENT_STR:
            if CHAR == ',' and PARENTHESIS_COUNT == 0:
                ARGS.append(CURRENT_ARG.strip())
                CURRENT_ARG = ""

            else:
                CURRENT_ARG += CHAR
                if CHAR == '(':         PARENTHESIS_COUNT += 1
                elif CHAR == ')':       PARENTHESIS_COUNT -= 1

        if CURRENT_ARG:                 ARGS.append(CURRENT_ARG.strip())
        
        return ARGS




    def NONE_PREFIX(self, VALUE):

        REFERENCE_PATTERN   = r"^[A-Za-z]+\d+$"
        RANGE_PATTERN       = r"^[A-Za-z]+:[A-Za-z]+$"
        
        if re.match(REFERENCE_PATTERN, VALUE) or re.match(RANGE_PATTERN, VALUE):

            if "!" not in VALUE:
                return f"NONE!{VALUE}"
            
        return VALUE



    def PROCESS_CONDITIONS(self, CONDITIONS):

        CONDITION_PARTS     = re.split(r'(==|!=|<=|>=|<|>|=)', CONDITIONS)
        PROCESSED_PARTS     = [self.NONE_PREFIX(part.strip()) for part in CONDITION_PARTS]
        
        return ''.join(PROCESSED_PARTS)





In [59]:
class ADD_REFERENCES_TO_EXTRACTOR:

    def __init__(self, extractor_dict, cross_ref_dict):
        self.EXTRACTOR_DICT = extractor_dict
        self.CROSS_REF_DICT = cross_ref_dict

    def ADD_REFERENCES(self):
        for SHEET_NAME, CELLS in self.EXTRACTOR_DICT.items():
            for CELL_REF, CELL_DATA in CELLS.items():

                REFERENCES = []
                if SHEET_NAME in self.CROSS_REF_DICT and CELL_REF in self.CROSS_REF_DICT[SHEET_NAME]:
                    REFERENCES = self.CROSS_REF_DICT[SHEET_NAME][CELL_REF]
                
                CELL_DATA['REFERENCES'] = REFERENCES
                

In [75]:

FUNC_PATH           = r'/Users/westhomas/Desktop/ALFRED/1__REPORT_AUTOMATION/USEFUL_FUNCTIONS/FORMULA_API/REFERENCE_FILES/FUNCTIONS.xlsx'

EXTRACTOR           = EXTRACT_FORMULAS(WORKSHEETS)
FORMULA             = FORMULA_ANALYZER(EXTRACTOR.DICT, FUNC_PATH)

FORMULA.ANALYSE_FORMULAS() 

CROSS_REF           = CROSS_SHEET_REFERENCE_IDENTIFIER(EXTRACTOR.DICT)
REFERENCES          = ADD_REFERENCES_TO_EXTRACTOR(EXTRACTOR.DICT, CROSS_REF.DICT)

REFERENCES.ADD_REFERENCES()




In [64]:
# Output the final data including the formulas with their breakdowns and references

In [76]:
EXTRACTOR.DICT  # Contains the formulas and their breakdowns

{'INPUT_2': {},
 'INPUT_1': {'C2': {'FORMULA': 'IF(A2<>B2,VLOOKUP(A2,INPUT_2!A:B,2,0),VLOOKUP(B2,INPUT_2!A:B,2,0))',
   'ERROR': False,
   'FORMULA_BREAKDOWN': {'FUNCTION': 'IF',
    'COMPONENTS': {'CONDITION': 'NONE!A2!=NONE!B2',
     'TRUE': {'FUNCTION': 'VLOOKUP',
      'COMPONENTS': {'LOOKUP_VALUE': 'NONE!A2',
       'TABLE_ARRAY': 'INPUT_2!A:B',
       'COL_INDEX_NUM': '2',
       'RANGE_LOOKUP': '0'}},
     'FALSE': {'FUNCTION': 'VLOOKUP',
      'COMPONENTS': {'LOOKUP_VALUE': 'NONE!B2',
       'TABLE_ARRAY': 'INPUT_2!A:B',
       'COL_INDEX_NUM': '2',
       'RANGE_LOOKUP': '0'}}}},
   'REFERENCES': [{'SHEET': 'INPUT_1', 'CELL': 'A2'},
    {'SHEET': 'INPUT_1', 'CELL': 'B2'},
    {'SHEET': 'INPUT_1', 'CELL': 'A2'},
    {'SHEET': 'INPUT_2', 'CELL': 'A:B'},
    {'SHEET': 'INPUT_1', 'CELL': 'B2'},
    {'SHEET': 'INPUT_2', 'CELL': 'A:B'}]}}}